# Homework 1
Consider the unconstrained problem
$$
\min f(x), \hspace{0.5 cm} \text{where  } f(x) = \left\{100(x_2-x_1)^2+(1-x_1)^2\right\}
$$
whose exact minimum is at $x^∗ = (1, 1)$. Find an estimate of $x^∗$ using:
* [a gradient method](#Gradient-descent),
* [a Newton's method](#newtons-method),
* [a conjugate direction method](#conjugate-direction-method).

In all cases use $x_0=(2,5)$ as starting point, backtracking line search with $\alpha=0.25$ and $\beta=0.5$, and $\|\nabla f(x_k)\|<10^{-5}$ as a stopping criterium. How many iterations have you used to find the estimate of $x^∗$ with these methods?

In [17]:
import numpy as np
from numpy.typing import ArrayLike
import numpy.linalg as npl


# Answer
First we define some functions that we are going to use in this homework. These are the function we want to minimize, its gradient and the line search method.

In [18]:
X = np.array([2,5]) # the initial point


def f(x: ArrayLike) -> float:
    """The function that we want to minimize"""
    assert len(x) == 2, "Size doesn't match, the size must be 2."
    return 100*(x[1]-x[0])**2 + (1-x[0])**2


def gradient(x: ArrayLike) -> ArrayLike:
    """Returns the value of the gradient at the given point"""
    assert len(x) == 2, "Size doesn't match, the size must be 2."
    return np.array([-2*(1-x[0]) - 200*(x[1]-x[0]), 200*(x[1]-x[0])])


def line_search(d_k: ArrayLike, x: ArrayLike) -> float:
    """Implement the line search method, this function update the time step
    until the criterion is fulfilled"""

    alpha = 0.25
    beta = 0.5
    t = 1
    while f(x + t*d_k) > f(x) + alpha * t * np.dot(gradient(x),d_k):
        t *= beta
    return t

    


The three methods that we have to implement differ only in the way we define our descend direction. The first one is gradient descent.

## Gradient descent

In this method we set our descent direction $d_k = -\nabla f(x_k)$.

As a stopping criterium we set $\|\nabla f(x_k)\|<10^{-5}$ but have to put another criterium using the number of iterations in case that the other stopping criterium is never achieved. To compute the norm of the gradient we make a dot product. 


In [19]:
def gradient_descend(stop: float, x: ArrayLike) -> tuple[ArrayLike, int]:
    """The gradient descend method using line search"""

    
    grad = gradient(x)
    if np.dot(grad, grad) < stop**2: #check the case that we are already in a minimum
        return x, 0
    
    for ite in range(1,10_000):
        descend_direction = -grad
        t = line_search(descend_direction, x)
        x = x + t * descend_direction
        grad = gradient(x)
        if np.dot(grad, grad) < stop**2: 
            return x, ite
        
    print("The stop criterion wasn't achieve in 10000 iterations.")
    return x, ite
   
    

In [28]:
mini, iterations = gradient_descend(1.0e-05, X)
print(f"The minimum is achieve at {mini} using {iterations} iterations")


The minimun is achieve at [1.0000059  1.00000595] using 2221 iterations


We can see that we achieve almost an exact minimum. We compute the method more than 2000 times which is a lot. We can compute the amount of time this method takes to execute and we see that it's around 190 ms which is not a lot. 

In [21]:
%%timeit
gradient_descend(1.0e-05, X)

198 ms ± 17.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Newton's method

In this case we define the descend direction as $d_k = -(\nabla^2f(x_k))^{-1}\nabla f(x_k)$. For this method we need the Hessian matrix $(\nabla^2f(x_k))$, so we need to compute the derivatives of the functions.
$$
\frac{\partial f}{\partial x_1} = -200 (x_2-x_1)-2 (1-x_1), 
$$
$$
\frac{\partial f}{\partial x_2} = 200 (x_2-x_1).
$$
From these expressions we can see that the second derivatives are constant, so the Hessian matrix for every $x_k$ is:
$$
\nabla^2f(x)=\begin{pmatrix} 202 & -200 \\ -200 & 200\end{pmatrix}.
$$
To compute the descend direction we need the inverse matrix so we are going to compute it:
$$
(\nabla^2f(x))^{-1} = \begin{pmatrix}1/2 & 1/2 \\ 1/2 & 101/200 \end{pmatrix}.
$$

The implementation of this method is the same as the previous one, we only have to change the descend direction. Because computing the inverse of the Hessian (solving a linear system) is part of the method we are going to do it using python, so when we compute the time execution time we take it into account.

In [22]:
HESSIAN = np.array([[202, -200], [-200, 200]])

def newtons_method(stop: float, x: ArrayLike) -> tuple[ArrayLike, int]:
    """Implementation of the newton method"""

    grad = gradient(x)
    if np.dot(grad, grad) < stop**2:  # check the case that we are already in a minimum
        return x, 0

    for ite in range(1,10_000):
        descend_direction = npl.solve(-HESSIAN, grad)
        t = line_search(descend_direction, x)
        x = x + t * descend_direction
        grad = gradient(x)
        if np.dot(grad, grad) < stop**2:
            return x, ite

    print("The stop criterium wasn't achieve in 10000 iterations.")
    return x, ite


In [23]:
mini, iterations = newtons_method(1.0e-05, X)
print(f"The minimum is achieve at {mini} using {iterations} iterations")


The minimun is achieve at [1. 1.] using 1 iterations


This second method is more accurate than the previous one because we get the exact solution. This solution is achieve only after one iteration of the method. The downside of the method is the need to calculate the inverse of a matrix, which we have done solving a system of equations instead of the direct computation of the matrix. This last part may be the reason of such a small time, 40.5 $\mu s$ on average.

In [24]:
%%timeit
newtons_method(1.0e-05, X)


44.4 µs ± 5.39 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Conjugate direction method
In this case we don't want to compute a matrix inversion. The implementation is the second one we saw in class.

In [25]:
def conjugate_direction(stop: float, x: ArrayLike) -> tuple[ArrayLike, int]:
    """Descend method with conjugate direction"""

    g_k = gradient(x)
    if np.dot(g_k, g_k) < stop**2:  # check the case that we are already in a minimum
        return x, ite
    A = HESSIAN
    b = np.dot(A,x) - g_k
    descend_direction = -g_k

    for ite in range(1,10_000):
        alpha = np.dot(g_k, descend_direction)/(npl.multi_dot([descend_direction,A,descend_direction]))
        x = x - alpha * descend_direction
        g_k = np.dot(A,x) - b
        beta = npl.multi_dot([g_k,A,descend_direction])/npl.multi_dot([descend_direction,A,descend_direction])
        descend_direction = -g_k + beta * descend_direction

        grad = gradient(x)
        if np.dot(grad, grad) < stop**2:
            return x, ite

    print("The stop criterium wasn't achieve in 10000 iterations.")
    return x, ite

    

In [26]:
mini, iterations = conjugate_direction(1.0e-05, X)
print(f"The minimun is achieve at {mini} using {iterations} iterations")


The minimun is achieve at [1. 1.] using 2 iterations


With this method we also arrive to the exact solution but in this case usign two iterations. Although we didn't solve a system with this method the fact that we need two iterations may be an explanation on the amount of time that this method need, 118 $\mu s$ on average. Also this problem is a small one and solving a $2\times 2$ system may be faster than all the matrix multiplications that we need to do in this method.

In [27]:
%%timeit
conjugate_direction(1.0e-05, X)


123 µs ± 5.26 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
